## Project – Creating an automated trading system

#### Christian Karvonen IA-15 & Wilhelm Kinos IA-15

#### Imports:

In [1]:
import datetime as dt
import numpy as np
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader.data import DataReader
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.linear_model import LinearRegression, ElasticNetCV, Ridge, BayesianRidge, LassoCV, LassoLarsCV
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing, svm, model_selection
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
init_notebook_mode(connected=True)

### All definitions/functions:

#### Rolling average definition:

In [2]:
def rolling_average(values, window):
    nan = []
    nan = np.append(nan, np.repeat(np.nan, window-1))
    weights = np.repeat(1.0, window)/window
    sma = np.convolve(values,weights,'valid')
    sma = np.append(nan, sma)
    return sma

#### MASE

In [3]:
def mean_absolute_scaled_error(y_test, forecast_prediction):
    y_test, forecast_prediction = np.array(y_test), np.array(forecast_prediction)
    n = len(y_test)
    d = np.abs(np.diff(y_test, axis=0)).sum()/(n-1) 
    errors = np.abs(y_test.flat - forecast_prediction)
    return errors.mean()/d

#### SMAPE

In [4]:
def SMAPE(y_test, forecast_prediction):
    y_test, forecast_prediction = np.array(y_test), np.array(forecast_prediction)
    return 1/len(y_test) * np.sum(np.abs(forecast_prediction - y_test.flat) / (np.abs(y_test) + np.abs(forecast_prediction))/2)

#### Defining timeframe of stocks:

In [5]:
end = dt.date.today()
endday = end
endday = endday.strftime('%Y-%m-%d')
print([endday])
start = end - dt.timedelta(days=5*365)

['2018-10-31']


#### Use of DataReader to get MU stock data from IEX https://iextrading.com/developer/

In [6]:
df = DataReader('MU', 'iex', start, end)
df.tail(10)

,open,high,low,close,volume
date,,,,,
2018-10-17,43.33,43.3900,42.110,42.35,27758963
2018-10-18,42.04,42.2125,41.140,41.30,33560822
2018-10-19,41.67,41.9850,40.270,40.45,28641375
2018-10-22,40.56,40.7000,38.640,39.76,34441221
2018-10-23,38.15,39.1100,37.410,38.68,46364174
2018-10-24,38.00,38.3000,35.400,35.43,53096450
2018-10-25,35.59,37.1600,35.060,36.78,39625699
2018-10-26,35.28,35.9100,34.300,35.40,41257357
2018-10-29,36.14,36.2000,33.820,34.66,34176423


### The 5 features:

#### Momentum: 

In [7]:
df['momentum'] = df.close - df.close.shift(4)
df[['momentum']].head(10)

,momentum
date,
2013-11-01,NaN
2013-11-04,NaN
2013-11-05,NaN
2013-11-06,NaN
2013-11-07,0.144
2013-11-08,0.215
2013-11-11,0.600
2013-11-12,1.130
2013-11-13,1.201


#### A/O Oscillator (accumulation/distribution oscillator):

In [8]:
df['A/O'] = (df.high - df.close.shift(1))/(df.high - df.low)
df[['A/O']].head(10)

,A/O
date,
2013-11-01,NaN
2013-11-04,0.882979
2013-11-05,-0.333333
2013-11-06,0.348837
2013-11-07,1.139130
2013-11-08,0.786275
2013-11-11,0.460526
2013-11-12,0.727273
2013-11-13,0.631579


#### MA5:

In [9]:
df['MA5'] = rolling_average(df.close, 5)
df[['MA5']].head(10)

,MA5
date,
2013-11-01,NaN
2013-11-04,NaN
2013-11-05,NaN
2013-11-06,NaN
2013-11-07,17.6708
2013-11-08,17.7768
2013-11-11,17.8528
2013-11-12,18.0448
2013-11-13,18.3288


#### BIAS6:

In [10]:
df['MA6'] = rolling_average(df.close, 6)
df['BIAS6'] = ((df.close-df.MA6)/(df.MA6))*100
df[['BIAS6']].head(10)

,BIAS6
date,
2013-11-01,NaN
2013-11-04,NaN
2013-11-05,NaN
2013-11-06,NaN
2013-11-07,NaN
2013-11-08,2.039283
2013-11-11,2.301361
2013-11-12,3.601683
2013-11-13,4.009382


#### ROC (rate of change)

In [11]:
df['ROC'] = (df.close - df.close.shift(3)) / df.close.shift(3) * 100
df[['ROC']].head(10)

,ROC
date,
2013-11-01,NaN
2013-11-04,NaN
2013-11-05,NaN
2013-11-06,-0.426743
2013-11-07,-0.955841
2013-11-08,2.461800
2013-11-11,4.400000
2013-11-12,5.141374
2013-11-13,4.501519


### Moving window

In [12]:
def get_moving_window(data, window):
    for i in range(1, window + 1):
        df["Moving_Window_"+str(i)] = data.shift(i) 

In [13]:
get_moving_window(df.close, 10)
df.head(10)

,open,high,low,close,volume,momentum,A/O,MA5,MA6,BIAS6,...,Moving_Window_1,Moving_Window_2,Moving_Window_3,Moving_Window_4,Moving_Window_5,Moving_Window_6,Moving_Window_7,Moving_Window_8,Moving_Window_9,Moving_Window_10
date,,,,,,,,,,,,,,,,,,,,,
2013-11-01,17.75,17.910,17.43,17.575,41372306,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-04,17.62,17.990,17.52,17.890,37802791,NaN,0.882979,NaN,NaN,NaN,...,17.575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-05,17.74,17.790,17.49,17.670,23800232,NaN,-0.333333,NaN,NaN,NaN,...,17.890,17.575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-06,17.77,17.820,17.39,17.500,21799539,NaN,0.348837,NaN,NaN,NaN,...,17.670,17.890,17.575,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-07,17.83,18.155,17.58,17.719,54196790,0.144,1.139130,17.6708,NaN,NaN,...,17.500,17.670,17.890,17.575,NaN,NaN,NaN,NaN,NaN,NaN
2013-11-08,17.70,18.120,17.61,18.105,33201332,0.215,0.786275,17.7768,17.743167,2.039283,...,17.719,17.500,17.670,17.890,17.575,NaN,NaN,NaN,NaN,NaN
2013-11-11,18.04,18.280,17.90,18.270,36990366,0.600,0.460526,17.8528,17.859000,2.301361,...,18.105,17.719,17.500,17.670,17.890,17.575,NaN,NaN,NaN,NaN
2013-11-12,18.17,18.670,18.12,18.630,32232323,1.130,0.727273,18.0448,17.982333,3.601683,...,18.270,18.105,17.719,17.500,17.670,17.890,17.575,NaN,NaN,NaN
2013-11-13,18.47,18.990,18.42,18.920,38821218,1.201,0.631579,18.3288,18.190667,4.009382,...,18.630,18.270,18.105,17.719,17.500,17.670,17.890,17.575,NaN,NaN


Dropna

In [14]:
df.dropna(inplace=True)
df.head()

,open,high,low,close,volume,momentum,A/O,MA5,MA6,BIAS6,...,Moving_Window_1,Moving_Window_2,Moving_Window_3,Moving_Window_4,Moving_Window_5,Moving_Window_6,Moving_Window_7,Moving_Window_8,Moving_Window_9,Moving_Window_10
date,,,,,,,,,,,,,,,,,,,,,
2013-11-15,19.38,19.830,19.22,19.461,51592829,1.191,1.049180,18.8942,18.762667,3.721930,...,19.190,18.920,18.630,18.270,18.105,17.719,17.500,17.670,17.890,17.575
2013-11-18,19.58,19.880,19.09,19.290,47549892,0.660,0.530380,19.0982,18.960167,1.739612,...,19.461,19.190,18.920,18.630,18.270,18.105,17.719,17.500,17.670,17.890
2013-11-19,19.35,19.870,19.05,19.060,51853726,0.140,0.707317,19.1842,19.091833,-0.166738,...,19.290,19.461,19.190,18.920,18.630,18.270,18.105,17.719,17.500,17.670
2013-11-20,19.19,19.220,18.51,18.800,44815768,-0.390,0.225352,19.1602,19.120167,-1.674497,...,19.060,19.290,19.461,19.190,18.920,18.630,18.270,18.105,17.719,17.500
2013-11-21,19.04,20.115,18.96,19.990,93347344,0.529,1.138528,19.3202,19.298500,3.583180,...,18.800,19.060,19.290,19.461,19.190,18.920,18.630,18.270,18.105,17.719


Linear Regression

In [15]:
def get_y_columns(data):
    feature_col = 11
    return data.columns[feature_col:].values 
    
get_y_columns(df)

array(['Moving_Window_1', 'Moving_Window_2', 'Moving_Window_3',
       'Moving_Window_4', 'Moving_Window_5', 'Moving_Window_6',
       'Moving_Window_7', 'Moving_Window_8', 'Moving_Window_9',
       'Moving_Window_10'], dtype=object)

In [31]:
x = np.array(df.drop(get_y_columns(df), 1))
y = np.array(df[get_y_columns(df)])

y = y.reshape(-1, 1)

scaler = preprocessing.StandardScaler().fit(y)

x = scaler.transform(x)
y = scaler.transform(y)

tscv = TimeSeriesSplit(n_splits=5)

for train_index, test_index in tscv.split(x):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [32]:
regr = LinearRegression()
regr.fit(x_train, y_train.reshape(len(y_train)))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [33]:
regr.score(x_test, y_test)

-3.8841986600116689